# **RVC - Disconnected**

***Modified by [MedicDoesStuff](https://www.youtube.com/@medicdoesstuff/)***

**Original notebook hastily written by [Kit Lemonfoot](https://huggingface.co/Kit-Lemonfoot) / [Noel Shirogane's High Flying Birds](https://www.youtube.com/@NoelShiroganesHighFlyingBirds)**

*Based on the work of the [RVC Project](https://github.com/RVC-Project), [Mangio261](https://github.com/Mangio621/), [Kalomaze](https://github.com/kalomaze), the [Pony Preservation Project](https://boards.4channel.org/mlp/catalog#s=Pony%20Preservation%20Project), and many others in the RVC / voice AI community*

*Notebook version:* **0.19.a**

---

This notebook is designed to be used with training Retreival-Based Voice Conversion (RVC) models without using a WеbUl. This is done to stay within the scope of Colab's TOS.

⚠️ WARNING: Unlike the original RVC notebook, *this notebook ***does not*** have autosave functionality* due to the massive amount of underlying stress that RVC's autosave features places on Colab to Drive communication. Please be careful and try to keep training sessions short.

# **Dependencies**
🟢 Run this first!

In [ ]:
#@title Install Dependencies
import subprocess

packages = ['build-essential', 'python3-dev', 'ffmpeg', 'aria2']
pip_packages = ['pip', 'setuptools', 'wheel', 'httpx==0.23.0', 'faiss-gpu', 'fairseq', 'ffmpeg', 'ffmpeg-python', 'praat-parselmouth', 'pyworld', 'numpy==1.23.5', 'numba==0.56.4', 'librosa==0.9.2', 'gdown', 'onnxruntime']
print("Updating and installing system packages...")
for package in packages:
  print(f"Installing {package}...")
  subprocess.check_call(['apt-get', 'install', '-qq', '-y', package])

print("Updating and installing pip packages...")
subprocess.check_call(['pip', 'install', '--upgrade'] + pip_packages)

print('Packages up to date.')
firsttry = True

In [ ]:
#@title Clone Repositories
import os

os.chdir('/content/')
!git clone https://github.com/Mangio621/Mangio-RVC-Fork.git
!git clone https://github.com/maxrmorrison/torchcrepe.git
!mv torchcrepe/torchcrepe Mangio-RVC-Fork/
!rm -rf torchcrepe  # Delete the torchcrepe repository folder
os.chdir('/content/Mangio-RVC-Fork')

now_dir = "/content/Mangio-RVC-Fork"
os.makedirs(os.path.join(now_dir, "logs"), exist_ok=True)
os.makedirs(os.path.join(now_dir, "weights"), exist_ok=True)


In [ ]:
#@title GPU Check
import torch

ngpu = torch.cuda.device_count()
gpu_infos = []
mem = []
if_gpu_ok = False

if torch.cuda.is_available() or ngpu != 0:
  for i in range(ngpu):
    gpu_name = torch.cuda.get_device_name(i)
    if any(
        value in gpu_name.upper()
        for value in ["10", "16", "20", "30", "40", "A2", "A3", "A4", "P4", "A50", "500", "A60", "70", "80", "90", "M4", "T4", "TITAN"]
    ):
      if_gpu_ok = True
      print("Compatible GPU detected: %s" % gpu_name)
      gpu_infos.append("%s\t%s" % (i, gpu_name))
      mem.append(int(torch.cuda.get_device_properties(i).total_memory / 1024 / 1024 / 1024 + 0.4))

if if_gpu_ok and len(gpu_infos) > 0:
  gpu_info = "\n".join(gpu_infos)

else:
  raise Exception("No GPU detected; training cannot continue. Please change your runtime type to a GPU.")
gpus = "-".join(i[0] for i in gpu_infos)

In [ ]:
#@title Mount Drive
from google.colab import drive
if not os.path.exists('/content/drive'):
  drive.mount('/content/drive')
else:
  print('Drive is already mounted. Proceed.')

os.makedirs('/content/drive/MyDrive/RVC-Disconnected', exist_ok=True)

In [ ]:
#@title Download Pretrained Models
#Didn't ask.

#V1 Models
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/v1/f0G32k.pth -d /content/Mangio-RVC-Fork/pretrained -o f0G32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/v1/f0D32k.pth -d /content/Mangio-RVC-Fork/pretrained -o f0D32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/v1/f0G40k.pth -d /content/Mangio-RVC-Fork/pretrained -o f0G40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/v1/f0D40k.pth -d /content/Mangio-RVC-Fork/pretrained -o f0D40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/v1/f0G48k.pth -d /content/Mangio-RVC-Fork/pretrained -o f0G48k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/v1/f0D48k.pth -d /content/Mangio-RVC-Fork/pretrained -o f0D48k.pth

#V2 Models
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/f0G32k.pth -d /content/Mangio-RVC-Fork/pretrained_v2 -o f0G32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/f0D32k.pth -d /content/Mangio-RVC-Fork/pretrained_v2 -o f0D32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/f0G40k.pth -d /content/Mangio-RVC-Fork/pretrained_v2 -o f0G40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/f0D40k.pth -d /content/Mangio-RVC-Fork/pretrained_v2 -o f0D40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/f0G48k.pth -d /content/Mangio-RVC-Fork/pretrained_v2 -o f0G48k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/f0D48k.pth -d /content/Mangio-RVC-Fork/pretrained_v2 -o f0D48k.pth

#RMVPE and Hubert
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/hubert_base.pt -d /content/Mangio-RVC-Fork -o hubert_base.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/rmvpe.pt -d /content/Mangio-RVC-Fork -o rmvpe.pt

#fp_16 Variant JSONs
!rm -rf /content/Mangio-RVC-Fork/configs/32k.json
!rm -rf /content/Mangio-RVC-Fork/configs/40k.json
!rm -rf /content/Mangio-RVC-Fork/configs/48k.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/32k.json -d /content/Mangio-RVC-Fork/configs -o 32k.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/40k.json -d /content/Mangio-RVC-Fork/configs -o 40k.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/48k.json -d /content/Mangio-RVC-Fork/configs -o 48k.json

In [ ]:
#@title Setup CSVDB
#...Alright, you made your point.
import csv

if not os.path.isdir("csvdb/"):
  os.makedirs("csvdb")
  frmnt, stp = open("csvdb/formanting.csv", "w", newline=""), open("csvdb/stop.csv", "w", newline="")
  csv_writer = csv.writer(frmnt, delimiter=",")
  csv_writer.writerow([False, 1.0, 1.0])
  csv_writer = csv.writer(stp, delimiter=",")
  csv_writer.writerow([False])
  frmnt.close()
  stp.close()

global DoFormant, Quefrency, Timbre
DoFormant, Quefrency, Timbre = False, 1.0, 1.0

# **Set Training Variables**

- `experiment_name`: The name of the model. (i.e: EminemKamikaze)
- `model_architecture`: Specifies the chosen model version. V2 is recommended.
- `target_sample_rate`: Specifies the desired sample rate for the model, 40K is recommended for general training.
- `cpu_threads`: Specifies the number of CPU threads to be used during the training. Colab only has 2 so don't even bother changing it.
- `speaker_id`: Represents the ID of the speaker being trained, I recommend not changing it.
- `pitch_extraction_algorithm`: Specifies the algorithm used for pitch extraction from the audio data, rmvpe is recommended.
- `crepe_hop_length`: Represents the hop length parameter used in the Crepe algorithm for pitch extraction, only affects crepe and mangio-crepe.
- `pitch_guidance`: Indicates whether pitch guidance is enabled or disabled for the experiment, can be useful if you want a talking or singing model.

In [ ]:
#title Set Training Variables (Run Every Time)
now_dir = "/content/Mangio-RVC-Fork"
experiment_name = "experiment_name" #@param {type:"string"}
path_to_training_folder = "/content/dataset/"
model_architecture = "v2" #@param ["v1","v2"] {allow-input: false}
target_sample_rate = "40k" #@param ["32k", "40k", "48k"] {allow-input: false}
cpu_threads = 2 #@param {type:"integer"}
speaker_id = 0 #@param {type:"integer"}
pitch_extraction_algorithm = "rmvpe" #@param ["harvest", "crepe", "mangio-crepe", "rmvpe"] {allow-input: false}
crepe_hop_length = 64 #@param {type:"slider", min:1, max:512, step:1}
pitch_guidance = True #@param {type:"boolean"}

if(experiment_name == "experiment_name"):
  print("Warning: Your experiment name should be changed to the name of your dataset.")

if(experiment_name.find(" ")!=-1):
  experiment_name = experiment_name.replace(" ", "_")
  print("I detected spaces in your experiment name, which should not be used. These spaces have been replaced with underscores.")
  print("Your new experiment name is: "+experiment_name)

#**Preprocessing**
You should only need to run these cells once per model.

In [ ]:
#@title Load dataset
#@markdown If it doesn't already exist, create a folder in your Google Drive named 'RVC-Disconnected' and place your zip file there. This will look for the following ZIP file inside that 'RVC-Disconnected' folder.
dataset = "zipfile.zip"  #@param {type:"string"}

#@markdown This loader will load datasets in a similar fashion to the So-Vits-SVC dataset loader. For best results, your dataset should be formatted as such:
#@markdown ```
#@markdown zipfile.zip
#@markdown └───character_name
#@markdown     ├───file1.wav
#@markdown     ├───...
#@markdown     └───file999.wav
#@markdown ```
#@markdown Audio filenames do not matter. All audio files should be in WAV format for best compatibility.

# TODO: Add something to convert non-WAVs to WAV

import os
import shutil

directories=[]

def sanitize_directory(directory):
  for filename in os.listdir(directory):
    file_path = os.path.join(directory, filename)
    if os.path.isfile(file_path):
      if filename == ".DS_Store" or filename.startswith("._") or not filename.endswith(('.wav', '.flac', '.mp3', '.ogg', '.m4a')):
        os.remove(file_path)
    elif os.path.isdir(file_path):
      #Get rid of the MACOSX directory just so it doesn't mess with renaming later
      if(filename == "__MACOSX"):
        shutil.rmtree(file_path)
        continue
      #Append the directory to directories for future dataset check, then recurse.
      directories.append(file_path)
      sanitize_directory(file_path)

dataset_path = '/content/drive/MyDrive/RVC-Disconnected/' + dataset
final_directory = '/content/dataset'
temp_directory = '/content/temp_dataset'

if os.path.exists(final_directory):
  print("Dataset folder already found. Wiping...")
  shutil.rmtree(final_directory)
if os.path.exists(temp_directory):
  print("Temporary folder already found. Wiping...")
  shutil.rmtree(temp_directory)

if not os.path.exists(dataset_path):
  raise Exception(f'I can\'t find {dataset} in {os.path.dirname(dataset_path)}.')

os.makedirs(final_directory, exist_ok=True)
os.makedirs(temp_directory, exist_ok=True)
#Oops.
!unzip -d {temp_directory} -B "{dataset_path}"
print("Sanitizing...")
sanitize_directory(temp_directory)

if(len(directories) == 0):
  #If there's no directories, we're dealing with a ZIP of just audio files.
  #Move everything to /dataset/experiment_name/.
  print("Dataset Type: Audio Files (Single Speaker)")
  expDir=os.path.join(final_directory, experiment_name)
  os.makedirs(expDir, exist_ok=True)
  for r, _, f in os.walk(temp_directory):
    for name in f:
      !cp {temp_directory}/{name} {expDir}
elif(len(directories) == 1):
  #If there's only one directory, we're dealing with a single speaker.
  #Rename the folder to experiment_name and move it to /dataset/.
  print("Dataset Type: Single Speaker")
  fi = os.path.join(temp_directory, experiment_name)
  os.rename(directories[0], fi)
  shutil.move(fi, final_directory)

else:
  #If anything else, we're dealing with multispeaker.
  #Move all folders to /dataset/ indiscriminately.
  print("Dataset Type: Multispeaker")
  for fi in directories:
    shutil.move(fi, final_directory)

shutil.rmtree(temp_directory)

print("Dataset imported.")


In [ ]:
#@title Preprocessing
# Change the Experiment Name and the Path to Training Folder. You shouldn't need to change anything else.

import os
import subprocess

assert cpu_threads>0, "CPU threads not allocated correctly."

sr = int(target_sample_rate.rstrip('k'))*1000
pttf = path_to_training_folder + experiment_name
os.makedirs("%s/logs/%s" % (now_dir, experiment_name), exist_ok=True)

cmd = "python trainset_preprocess_pipeline_print.py %s %s %s %s/logs/%s 1" % (pttf, sr, cpu_threads, now_dir, experiment_name)
print(cmd)
!$cmd

In [ ]:
#@title Feature extraction

pitch_extraction_algorithm = "rmvpe" #@param ["harvest", "crepe", "mangio-crepe", "rmvpe"] {allow-input: false}
#crepe_hop_length = 128 #@param {type:"slider", min:1, max:512, step:1}
#pitch_guidance = True #@param {type:"boolean"}

gpuList = gpus.split("-")
cmd = "python extract_f0_print.py %s/logs/%s %s %s %s" % (now_dir, experiment_name, cpu_threads, pitch_extraction_algorithm, crepe_hop_length)
print(cmd)
!$cmd

leng = len(gpus)

cmd = "python extract_feature_print.py %s %s %s %s %s/logs/%s %s" % ("device", leng, 0, 0, now_dir, experiment_name, model_architecture)
print(cmd)
!$cmd


In [ ]:
#@title Save preprocessed dataset files to Google Drive
#Compress dataset folder
loc = "%s/logs/%s" % (now_dir, experiment_name)
!zip -r rvcLogs.zip {loc}
DATASET_PATH_DRIVE = "/content/drive/MyDrive/RVC-Disconnected/" + experiment_name
!mkdir -p {DATASET_PATH_DRIVE}
!cp /content/Mangio-RVC-Fork/rvcLogs.zip {DATASET_PATH_DRIVE}

# **Training**
Also includes resuming code.

In [ ]:
#@title Load preprocessed dataset files from Google Drive (for resuming)
#@markdown If you already have preprocessed dataset files on Google Drive, you can load them here instead of re-running the preprocessing steps.
import os

BACK_UP_DATASET_PATH = "/content/drive/MyDrive/RVC-Disconnected/" + experiment_name

#Prevent people from loading the ZIP over existing files
ok=True
if(os.path.exists("/content/Mangio-RVC-Fork/logs/"+experiment_name+"/2a_f0")):
  print("Dataset files already loaded, skipping.")
  ok=False

if ok:
  !unzip {BACK_UP_DATASET_PATH}/rvcLogs.zip -d /

In [ ]:
#@title Import Model from Drive to Notebook (for resuming)
import os

DATASET_PATH_DRIVE = "/content/drive/MyDrive/RVC-Disconnected/" + experiment_name
DATASET_PATH_COLAB = "/content/Mangio-RVC-Fork/logs/" + experiment_name
#@markdown Input the model's Step Count here (the number located on your model's G and D files.) If you used `save_only_last_ckpt` during training, this number will be 2333333.
STEPCOUNT = 2333333 #@param {type:"integer"}

print("Copying model files...")
!cp {DATASET_PATH_DRIVE}/D_{STEPCOUNT}.pth {DATASET_PATH_COLAB}
!cp {DATASET_PATH_DRIVE}/G_{STEPCOUNT}.pth {DATASET_PATH_COLAB}
!cp {DATASET_PATH_DRIVE}/config.json {DATASET_PATH_COLAB}

print("Copying Tensorboard TFEVENT files...")
for r, _, f in os.walk(DATASET_PATH_DRIVE):
  for name in f:
    if(name.startswith("events.out.tfevents")):
      !cp {DATASET_PATH_DRIVE}/{name} {DATASET_PATH_COLAB}

print("All done. Welcome back!")

In [ ]:
import os
import math
from random import shuffle

# I will not be adding an autosave feature. Do not ask.

#@title Training
save_frequency = 10 #@param {type:"slider", min:1, max:50, step:1}
total_epochs = 300 #@param {type:"slider", min:1, max:10000, step:1}
batch_size = 12 #@param {type:"slider", min:1, max:40, step:1}
save_only_latest_ckpt = True #@param {type:"boolean"}
cache_all_training_sets = False #@param {type:"boolean"}
save_small_final_model = True #@param {type:"boolean"}
#@markdown The automatically calculated log interval is known to be very inaccurate and can cause delays between an epoch finishing and Tensorboard writes. If you would like, you can manually define a log interval here.
use_manual_stepToEpoch = False #@param {type:"boolean"}
manual_stepToEpoch = 000 #@param {type:"integer"}

pretrained_base = "pretrained/" if model_architecture == "v1" else "pretrained_v2/"
exp_dir = "%s/logs/%s" % (now_dir, experiment_name)

pretrainedD = "%sf0D%s.pth" % (pretrained_base, target_sample_rate)
pretrainedG = "%sf0G%s.pth" % (pretrained_base, target_sample_rate)

#Log interval
log_interval = 1
liFolderPath = os.path.join(exp_dir, "1_16k_wavs")
if(os.path.exists(liFolderPath) and os.path.isdir(liFolderPath)):
  wav_files = [f for f in os.listdir(liFolderPath) if f.endswith(".wav")]
  if wav_files:
    sample_size = len(wav_files)
    log_interval = math.ceil(sample_size / batch_size)
    if log_interval > 1:
      log_interval += 1

if log_interval > 250 and not use_manual_stepToEpoch:
  print("That's a big dataset you got there. Log interval normalized to 200 steps from %s steps." % log_interval)
  log_interval = 200

if use_manual_stepToEpoch:
  log_interval = manual_stepToEpoch

#Create Python command
cmd = "python train_nsf_sim_cache_sid_load_pretrain.py -e %s -sr %s -f0 %s -bs %s -g %s -te %s -se %s %s %s -l %s -c %s -sw %s -v %s -li %s" % (
    experiment_name,
    target_sample_rate,
    1,
    batch_size,
    0,
    total_epochs,
    save_frequency,
    "-pg %s" % pretrainedG if pretrainedG != "" else "\b",
    "-pd %s" % pretrainedD if pretrainedD != "" else "\b",
    1 if save_only_latest_ckpt else 0,
    1 if cache_all_training_sets else 0,
    1 if save_small_final_model else 0,
    model_architecture,
    log_interval,
)
print(cmd)

#Create mute filelist
exp_dir = "%s/logs/%s" % (now_dir, experiment_name)
gt_wavs_dir = "%s/0_gt_wavs" % (exp_dir)
feature_dir = (
  "%s/3_feature256" % (exp_dir)
  if model_architecture == "v1"
  else "%s/3_feature768" % (exp_dir)
)
f0_dir = "%s/2a_f0" % (exp_dir)
f0nsf_dir = "%s/2b-f0nsf" % (exp_dir)
names = (
  set([name.split(".")[0] for name in os.listdir(gt_wavs_dir)])
  & set([name.split(".")[0] for name in os.listdir(feature_dir)])
  & set([name.split(".")[0] for name in os.listdir(f0_dir)])
  & set([name.split(".")[0] for name in os.listdir(f0nsf_dir)])
)
opt = []
for name in names:
  opt.append(
    "%s/%s.wav|%s/%s.npy|%s/%s.wav.npy|%s/%s.wav.npy|%s"
    % (
      gt_wavs_dir.replace("\\", "\\\\"),
      name,
      feature_dir.replace("\\", "\\\\"),
      name,
      f0_dir.replace("\\", "\\\\"),
      name,
      f0nsf_dir.replace("\\", "\\\\"),
      name,
      speaker_id,
    )
  )
fea_dim = 256 if model_architecture == "v1" else 768
for _ in range(2):
  opt.append(
      "%s/logs/mute/0_gt_wavs/mute%s.wav|%s/logs/mute/3_feature%s/mute.npy|%s/logs/mute/2a_f0/mute.wav.npy|%s/logs/mute/2b-f0nsf/mute.wav.npy|%s"
      % (now_dir, target_sample_rate, now_dir, fea_dim, now_dir, now_dir, speaker_id)
  )
shuffle(opt)
with open("%s/filelist.txt" % exp_dir, "w") as f:
  f.write("\n".join(opt))
print("Mute filelist written. Best of luck training!")


%cd /content/Mangio-RVC-Fork
%load_ext tensorboard
%tensorboard --logdir /content/Mangio-RVC-Fork/logs

os.chdir('/content/Mangio-RVC-Fork')
!$cmd


In [ ]:
#@title Export Model from Notebook to Drive
import os

DATASET_PATH_DRIVE = "/content/drive/MyDrive/RVC-Disconnected/" + experiment_name
DATASET_PATH_COLAB = "/content/Mangio-RVC-Fork/logs/" + experiment_name
if(not os.path.exists(DATASET_PATH_DRIVE)):
  !mkdir -p {DATASET_PATH_DRIVE}

#@markdown Use this option if you wish to only copy over weights.
skip_models = False #@param {type:"boolean"}
#@markdown Use these options if you wish to manually input your step count and epoch count for incomplete models. *Do not use this option if your training finished.*
manual_save = False #@param {type:"boolean"}
STEPCOUNT = 000 #@param {type:"integer"}
EPOCHCOUNT = 000 #@param {type:"integer"}

finished=False
potential="/content/Mangio-RVC-Fork/weights/"+experiment_name+".pth"
if os.path.exists(potential):
  finished = True

#VERY hacky. Might break stuff, report to me if it does.
print("Detecting latest model...")
if(not manual_save):
  currentMax = 0
  for r, _, f in os.walk("/content/Mangio-RVC-Fork/weights/"):
    for name in f:
      if(name.endswith(".pth") and (name!=experiment_name+".pth")):
        #Check to see if this PTH is what we're looking for.
        if(name.find(experiment_name)==-1):
          continue
        #Determine Epochcount+Stepcount Phase 1
        pot = name.split('_')
        ep=pot[len(pot)-2][1:]
        #If what we got from the epoch count section of the filename isn't a number, multiple completed models are in weights.
        #Skip it if that happens.
        if(not ep.isdecimal()):
          continue
        #Determine Epochcount+Stepcount Phase 2
        ep=int(ep)
        if ep>currentMax:
          currentMax=ep
          step=pot[len(pot)-1].split('.')
          step=int(step[0][1:])
          EPOCHCOUNT=ep
          STEPCOUNT=step

TSTEP = STEPCOUNT
if(not skip_models):
  print("Copying model files...")
  if(save_only_latest_ckpt):
    TSTEP=2333333
  !cp {DATASET_PATH_COLAB}/D_{TSTEP}.pth "{DATASET_PATH_DRIVE}"
  !cp {DATASET_PATH_COLAB}/G_{TSTEP}.pth "{DATASET_PATH_DRIVE}"
  !cp {DATASET_PATH_COLAB}/config.json "{DATASET_PATH_DRIVE}"

print("Copying Tensorboard TFEVENT files...")
for r, d, f in os.walk(DATASET_PATH_COLAB):
  for name in f:
    if(name.startswith("events.out.tfevents") and os.path.exists(os.path.join(DATASET_PATH_COLAB, name))):
      !cp {DATASET_PATH_COLAB}/{name} {DATASET_PATH_DRIVE}

print("Copying weight file...")
if(finished):
  !cp {potential} "{DATASET_PATH_DRIVE}"
else:
  !cp /content/Mangio-RVC-Fork/weights/{experiment_name}_e{EPOCHCOUNT}_s{STEPCOUNT}.pth "{DATASET_PATH_DRIVE}"

print("All done!")

In [ ]:
#@title Index training
#@markdown Ensure that Feature Extraction has run successfully before running this cell.

#@markdown **Note: It's rare, but you may encounter a bug with this cell that requires a runtime restart.**
#@markdown **If this happens, restart the runtime, re-run the "Set Training Variables" cell, then re-run this cell.**

#@markdown Use this option if you wish to save the two extra files generated by index training to your Google Drive. (Only the added index is normally needed.)
save_extra_files_to_drive = False #@param {type:"boolean"}

#Oh dear lord why is this baked into infer-web I hate this
import os
import sys
import traceback
import numpy as np
import faiss

#from sklearn.cluster import MiniBatchKMeans

exp_dir = "%s/logs/%s" % (now_dir, experiment_name)
os.makedirs(exp_dir, exist_ok=True)
feature_dir = (
    "%s/3_feature256" % (exp_dir)
    if model_architecture == "v1"
    else "%s/3_feature768" % (exp_dir)
)
print(feature_dir)
if not os.path.exists(feature_dir):
  raise Exception("No features exist for this model yet. Did you run Feature Extraction?")
listdir_res = list(os.listdir(feature_dir))
if len(listdir_res) == 0:
  raise Exception("No features exist for this model yet. Did you run Feature Extraction?")

try:
  from sklearn.cluster import MiniBatchKMeans
except:
  print("Due to a bug with Colab, we will need to reinstall Numpy real quick. Give me a sec!")
  !pip install -U numpy
  print("Numpy reinstalled. Please restart the runtime, and then re-run the \"Set Training Variables\" cell to continue.")
  sys.exit()
else:
  print("Proper Numpy version detected.")

infos=[]
npys=[]
for name in sorted(listdir_res):
  phone = np.load("%s/%s" % (feature_dir, name))
  npys.append(phone)
big_npy = np.concatenate(npys, 0)
big_npy_idx = np.arange(big_npy.shape[0])
np.random.shuffle(big_npy_idx)
if big_npy.shape[0] > 2e5:
  print("Trying doing kmeans %s shape to 10k centers." % big_npy.shape[0])
  try:
    big_npy = (
        MiniBatchKMeans(
            n_clusters=10000,
            verbose=True,
            batch_size=256,
            compute_labels = False,
            init="random"
        )
        .fit(big_npy)
        .cluster_centers_

    )
  except:
    info = traceback.format_exc()
    print(info)

np.save("%s/total_fea.npy" % exp_dir, big_npy)
n_ivf = min(int(16*np.sqrt(big_npy.shape[0])), big_npy.shape[0] // 39)
print("%s,%s" % (big_npy.shape, n_ivf))
index = faiss.index_factory(256 if model_architecture == "v1" else 768, "IVF%s,Flat" % n_ivf)
print("Training index...")
index_ivf = faiss.extract_index_ivf(index)
index_ivf.nprobe = 1
index.train(big_npy)
faiss.write_index(
    index,
    "%s/trained_IVF%s_Flat_nprobe_%s_%s_%s.index" % (exp_dir, n_ivf, index_ivf.nprobe, experiment_name, model_architecture)
)
print("Adding...")
batch_size_add = 8192
for i in range(0, big_npy.shape[0], batch_size_add):
  index.add(big_npy[i:i+batch_size_add])
faiss.write_index(
    index,
    "%s/added_IVF%s_Flat_nprobe_%s_%s_%s.index"
    % (exp_dir, n_ivf, index_ivf.nprobe, experiment_name, model_architecture)
)

npr = index_ivf.nprobe

print("Saving files to Drive...")
DATASET_PATH_DRIVE = "/content/drive/MyDrive/RVC-Disconnected/" + experiment_name
if(not os.path.exists(DATASET_PATH_DRIVE)):
  !mkdir -p {DATASET_PATH_DRIVE}
DATASET_PATH_COLAB = "/content/Mangio-RVC-Fork/logs/" + experiment_name
if(save_extra_files_to_drive):
  !cp {DATASET_PATH_COLAB}/total_fea.npy "{DATASET_PATH_DRIVE}"
  !cp {DATASET_PATH_COLAB}/trained_IVF{n_ivf}_Flat_nprobe_{npr}_{experiment_name}_{model_architecture}.index "{DATASET_PATH_DRIVE}"
!cp {DATASET_PATH_COLAB}/added_IVF{n_ivf}_Flat_nprobe_{npr}_{experiment_name}_{model_architecture}.index "{DATASET_PATH_DRIVE}"

print("All done! Your index file has completed training.")
try:
  firsttry
except:
  print("If you had to restart the runtime, disconnect and delete the runtime in order to continue. (Restarting the runtime again will not work.)")



# **TODO, Changelogs, Credits, and Special Thanks**
**TODO**
*   Continue to check for issues with the model exporter. The current non-manual method is very hacky and is prone to errors.
*   Add a WAV converter to the dataset loader
*   Implement "Export Lowest Points", or implement [ROD](https://github.com/grvyscale/RealtimeOvertrainingDetection) on a seperate thread
*   Remove some of the magic numbers from Feature Extraction (may not be needed, dunno if Colab has support for multiple GPUs anyways)

**Changelogs**

**v0.19** - Added some text to Import Model From Drive to better explain its' function.

Rearranged some cells to make resuming processes a little clearer.

Added a custom JSON import to force RVC to use fp16. *This may break stuff, report to me if it does.*

**v0.18** - Hotfix: Fixed multiple bugs with Export Model in regards to multiple completed models being in the Weights folder. Models that are not the experiment name are now ignored.

**v0.17** - Fixed a bug with Import Dataset that would fail to extract ZIP files with spaces in its' name.

Fixed a bug with Import Model that would attempt to look for the model in the wrong place. Oops.

**v0.16** - Fixed a bug with Export Model where models trained with `save_only_latest_ckpt` would not be saved to Drive properly.

Fixed a bug with Index Training and Export Model that glitched out saving if `experiment_name` folder did not exist on Drive yet from preprocess saving. These cells will now create the folder if it detects it does not exist.

Reverted a change in Export Model from 0.14 that prevented finished models from being saved.

**v0.15** - Reworked the CSVDB setup to remove the early CSVUtil reliancy. This should hopefully fix a rare bug that causes Colab to error out while creating the dummy formanting CSV files.

**v0.14** - Hotfix: Fixed an issue with Export Model where it would error out on model names with underscores. Hopefully.

**v0.13** - Hotfix: Added an experimental autodetector for Export Model.

Fixed a long-standing bug that would cause useless errors when transferring TFEVENT files out of a notebook.

**v0.12** - Reworked the dataset loader to accept more forms of datasets.

**v0.11** - Hotfix: Keep preprocessor ZIP loader from double-loads, added a warning regarding unchanged experiment_name, properly linked to the PPP in credits

**v0.1** - Initial release

**Credits**
*   [MedicDoesStuff](https://www.youtube.com/@medicdoesstuff/) - Modified the notebook
*   [Kit Lemonfoot](https://huggingface.co/Kit-Lemonfoot) - created the original No-WebUI notebook.
*   [RVC Project](https://github.com/RVC-Project) - Created RVC, obviously
*   [LJ1995](https://huggingface.co/lj1995) - Pretrained RVC models
*   [Mangio261](https://github.com/Mangio621/) - Creating the Mangio RVC fork
*   [Kalomaze](https://github.com/kalomaze) - Original RVC colab + Mangio tweaks
*   [Pony Preservation Project](https://boards.4channel.org/mlp/catalog#s=Pony%20Preservation%20Project) - for their robust TalkNet and So-Vits-SVC notebooks
*   the Colab team - forcing my hand and making me release this notebook early

**Special Thanks**
*   [Fifteen AI](https://15.ai/) - for getting me into voice AI in the first place
*   [Dacoolkid44](https://huggingface.co/dacoolkid44) / [HoloAI44](https://www.youtube.com/@Holo_AI44) and Hijack / [SANSSWEEP](https://huggingface.co/SANSSWEEP) - for basically kickstarting the larger VTuber voice AI scene with their models
*   [Maki Ligon](https://www.youtube.com/@Shiina_Mashiro) / [Yuuto Ichika](https://www.youtube.com/@yuutoichika) - for keeping me grounded in reality while developing this thing
*   [Bartezes](https://www.youtube.com/@bartezes3082) - for helping me so much with the [VTuber AI Model Tracking spreadsheet](https://docs.google.com/spreadsheets/d/1tvZSggOsZGAPjbMrWOAAaoJJFpJuQlwUEQCf5x1ssO8/)
*   [Megaaziib](https://www.youtube.com/@megaaziib) - for inspiring me to keep working on AI models and covers (I don't hate you)
*   [Saintlysaint](https://www.youtube.com/@SaintlySaint) and [Gengar2525](https://www.youtube.com/@GeGaCh) - for having faith in me


#**UVR Isolation Stuff**

In [ ]:
initialised = True
from time import sleep
from google.colab import output
from google.colab import drive

import sys
import os
import shutil
import psutil
import glob

mount_to_drive = True
mount_path = '/content/drive/MyDrive'

ai = 'https://github.com/kae0-0/Colab-for-MDX_B'
ai_version = 'https://github.com/kae0-0/Colab-for-MDX_B/raw/main/v'
onnx_list = 'https://raw.githubusercontent.com/kae0-0/Colab-for-MDX_B/main/onnx_list'
#@title Initialize UVR MDX-Net Models
#@markdown The 'ForceUpdate' option will update the models by fully reinstalling.
ForceUpdate = False #@param {type:"boolean"}
class h:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')
    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout
def get_size(bytes, suffix='B'): # read ram
    global svmem
    factor = 1024
    for unit in ["", "K", "M", "G", "T", "P"]:
        if bytes < factor:
            return f'{bytes:.2f}{unit}{suffix}'
        bytes /= factor
    svmem = psutil.virtual_memory()
def console(t):
    get_ipython().system(t)
def LinUzip(file): # unzip call linux, force replace
    console(f'unzip -o {file}')
#-------------------------------------------------------
def getONNX():
    console(f'wget {onnx_list} -O onnx_list')
    _onnx = open("onnx_list", "r")
    _onnx = _onnx.readlines()
    os.remove('onnx_list')
    for model in _onnx:
        _model = sanitize_filename(os.path.basename(model))
        console(f'wget {model}')
        LinUzip(_model)
        os.remove(_model)

def getDemucs(_path):
    #https://dl.fbaipublicfiles.com/demucs/v3.0/demucs_extra-3646af93.th
    root = "https://dl.fbaipublicfiles.com/demucs/v3.0/"
    model = {
        'demucs_extra': '3646af93'
    }
    for models in zip(model.keys(),model.values()):
        console(f'wget {root+models[0]+"-"+models[1]}.th -O {models[0]}.th')
    for _ in glob.glob('*.th'):
        if os.path.isfile(os.path.join(os.getcwd(),_path,_)):
            os.remove(os.path.join(os.getcwd(),_path,_))
        shutil.move(_,_path)

def mount(gdrive=False):
    if gdrive:
        if not os.path.exists("/content/drive/MyDrive"):
            try:
                drive.mount("/content/drive", force_remount=True)
            except:
                drive._mount("/content/drive", force_remount=True)
    else:
        pass

mount(mount_to_drive)

def toPath(path='local'):
    if path == 'local':
        os.chdir('/content')
    elif path == 'gdrive':
        os.chdir(mount_path)

def update():
    with h():
        console(f'wget {ai_version} -O nver')
    f = open('nver', 'r')
    nver = f.read()
    f = open('v', 'r')
    cver = f.read()
    if nver != cver or ForceUpdate:
        print('New update found! {}'.format(nver))
        os.chdir('../')
        print('Updating ai...',end=' ')
        with h():
            console(f'git clone {ai} temp_MDX_Colab')
            console('cp -a temp_MDX_Colab/* MDX_Colab/')
            console('rm -rf temp_MDX_Colab')
        print('done')
        os.chdir('MDX_Colab')
        print('Refreshing models...', end=' ')
        with h():
            #getDemucs('model/')
            getONNX()
        print('done')
        output.clear()
        os.remove('v')
        os.rename("nver",'v')
        #os.chdir(f'{os.path.join(mount_path,"MDX_Colab")}')
    else:
        os.remove('nver')
        print('Using latest version.')

def past_installation():
    return os.path.exists('MDX_Colab')

def LoadMDX():
    console(f'git clone {ai} MDX_Colab')

#-------------------------------------------------------
# install requirements
print('Installing dependencies will take 45 seconds...',end=' ')

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    svmem = psutil.virtual_memory()
    gpu_runtime = False
    with h():
        console('pip3 install onnxruntime==1.14.1')
else:
    gpu_runtime = True
    with h():
        console('pip3 install onnxruntime-gpu==1.14.1')
with h():
    deps = [
            'pathvalidate',
            'youtube-dl',
            'django'
    ]
    for dep in deps:
        console('pip3 install {}'.format(dep))
# import modules
#console('pip3 install torch==1.13.1')
console('pip3 install soundfile==0.12.1')
console('pip3 install librosa==0.9.1')
from pathvalidate import sanitize_filename
print('done')
if not gpu_runtime:
    print(f'GPU runtime is disabled. You have {get_size(svmem.total)} RAM.\nProcessing will be incredibly slow. 😈')
elif gpu_info.find('Tesla T4') >= 0:
    print('You got a Tesla T4 GPU. (speeds are around  10-25 it/s)')
elif gpu_info.find('Tesla P4') >= 0:
    print('You got a Tesla P4 GPU. (speeds are around  8-22 it/s)')
elif gpu_info.find('Tesla K80') >= 0:
    print('You got a Tesla K80 GPU. (This is the common gpu, speeds are around 2-10 it/s)')
elif gpu_info.find('Tesla P100') >= 0:
    print('You got a Tesla P100 GPU. (This is the Second to the fastest gpu, speeds are around  15-42 it/s)')
else:
    if gpu_runtime:
        print('You got an unknown GPU. Please report the GPU you got!')
        !nvidia-smi

#console('pip3 install demucs')
#-------------------------------------------------------
# Scripting
mount(mount_to_drive)
toPath('gdrive' if mount_to_drive else 'local')
#check for MDX existence
if not past_installation():
    print('First time installation will take around 3-6 minutes.\nThis requires around 2-3 GB Free Gdrive space.\nPlease try not to interup installation process!!')
    print('Downloading AI...',end=' ')
    with h():
        LoadMDX()
    os.chdir('MDX_Colab')
    print('done')

    print('Downloading models...',end=' ')
    with h():
        #getDemucs('model/')
        getONNX()
    if os.path.isfile('onnx_list'):
        os.remove('onnx_list')
    print('done')

else:
    os.chdir('MDX_Colab')
    update()

################
#outro
print('Success!')

##Audio Isolation

In [ ]:
#@markdown #Upload your files directly to UVR
#@markdown Run this cell to upload your vocal files that you want to use, (or zip files containing audio), to your Colab. <br>
#@markdown Alternatively, you can upload from the colab files panel, but this should be more convenient. This method may not work on iOS.

from google.colab import files
from IPython.display import display, Javascript
import os
import shutil
import zipfile
import ipywidgets as widgets

# Create the target directory if it doesn't exist
target_dir = '/content/drive/MyDrive/MDX_Colab/tracks'
if not os.path.exists(target_dir):
    os.makedirs(target_dir)

uploaded = files.upload()

for fn in uploaded.keys():
    # Check if the uploaded file is a zip file
    if fn.endswith('.zip'):
        # Write the uploaded zip file to the target directory
        zip_path = os.path.join(target_dir, fn)
        with open(zip_path, 'wb') as f:
            f.write(uploaded[fn])

        unzip_dir = os.path.join(target_dir, fn[:-4])  # Remove the .zip extension from the folder name

        # Extract the zip file
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(unzip_dir)

        # Delete the zip file
        if os.path.exists(zip_path):
            os.remove(zip_path)

        print('Zip file "{name}" extracted and removed. Files are in: {folder}'.format(name=fn, folder=unzip_dir))

        # Display copy path buttons for each extracted file
        for extracted_file in os.listdir(unzip_dir):
            extracted_file_path = os.path.join(unzip_dir, extracted_file)
            extracted_file_length = os.path.getsize(extracted_file_path)

            extracted_file_label = widgets.HTML(
                value='Extracted file "{name}" with length {length} bytes'.format(name=extracted_file, length=extracted_file_length)
            )
            display(extracted_file_label)

            extracted_file_path_text = widgets.HTML(
                value='File saved to: <a href="{}" target="_blank">{}</a>'.format(extracted_file_path, extracted_file_path)
            )

            extracted_copy_button = widgets.Button(description='Copy')
            extracted_copy_button_file_path = extracted_file_path  # Make a local copy of the file path

            def copy_to_clipboard(b):
                js_code = '''
                    const el = document.createElement('textarea');
                    el.value = "{path}";
                    el.setAttribute('readonly', '');
                    el.style.position = 'absolute';
                    el.style.left = '-9999px';
                    document.body.appendChild(el);
                    el.select();
                    document.execCommand('copy');
                    document.body.removeChild(el);
                '''
                display(Javascript(js_code.format(path=extracted_copy_button_file_path)))

            extracted_copy_button.on_click(copy_to_clipboard)
            display(widgets.HBox([extracted_file_path_text, extracted_copy_button]))

        continue

    # For non-zip files
    # Save the file to the target directory
    file_path = os.path.join(target_dir, fn)
    with open(file_path, 'wb') as f:
        f.write(uploaded[fn])

    file_length = len(uploaded[fn])
    file_label = widgets.HTML(
        value='User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=file_length)
    )
    display(file_label)

    # Check if the uploaded file is a .pth or .index file
    if fn.endswith('.pth') or fn.endswith('.index'):
        warning_text = widgets.HTML(
            value='<b style="color: red;">Warning:</b> You are uploading a model file in the wrong place. Please ensure it is uploaded to the correct location.'
        )
        display(warning_text)

    # Create a clickable path with copy button
    file_path_text = widgets.HTML(
        value='File saved to: <a href="{}" target="_blank">{}</a>'.format(file_path, file_path)
    )

    copy_button = widgets.Button(description='Copy')
    copy_button_file_path = file_path  # Make a local copy of the file path

    def copy_to_clipboard(b):
        js_code = '''
            const el = document.createElement('textarea');
            el.value = "{path}";
            el.setAttribute('readonly', '');
            el.style.position = 'absolute';
            el.style.left = '-9999px';
            document.body.appendChild(el);
            el.select();
            document.execCommand('copy');
            document.body.removeChild(el);
        '''
        display(Javascript(js_code.format(path=copy_button_file_path)))

    copy_button.on_click(copy_to_clipboard)
    display(widgets.HBox([file_path_text, copy_button]))

# Remove the original uploaded files from /content/
for fn in uploaded.keys():
    if os.path.exists(os.path.join("/content/", fn)):
        os.remove(os.path.join("/content/", fn))

In [ ]:
#@markdown ### Print a list of tracks
for i in glob.glob('tracks/*'):
    print(os.path.basename(i))

In [ ]:
if not 'initialised' in globals():
    raise NameError('Please run the first cell first!! #scrollTo=H_cTbwhVq4K6')

#import all models metadata
import json
with open('model_data.json', 'r') as f:
  model_data = json.load(f)

# Modifiable variables
tracks_path = 'tracks/'
separated_path = 'separated/'

#@markdown ### Input track
#@markdown Enter any link/Filename (Upload your songs in tracks folder)
track = "Eminem.wav" #@param {type:"string"}

#@markdown ---
#@markdown ### Models
ONNX = "MDX-UVR Vocal Model Voc_FT" #@param ["off", "Karokee", "Karokee 2", "Karokee_AGGR", "Kim ft other instrumental model", "MDX-UVR Ins Model 415", "MDX-UVR Ins Model 418", "MDX-UVR Ins Model 464", "MDX-UVR Ins Model 496 - inst main-MDX 2.1", "MDX-UVR Vocal Model 406", "MDX-UVR Ins Model Full Band 292", "MDX-UVR Ins Model Full Band 403", "MDX-UVR Ins Model Full Band 450 (HQ_1)", "MDX-UVR Ins Model Full Band 498 (HQ_2)", "MDX-UVR Vocal Model 427", "MDX-UVR Vocal Model 9662", "MDX-UVR Vocal Model 9682", "MDX-UVR Vocal Model 9703", "MDX-UVR-Kim Vocal Model (old)", "MDX-UVR-Kim Vocal Model 2 (Kim vocal 2)", "(de)Reverb HQ By FoxJoy", "MDX-UVR Ins Model Full Band new (HQ_3)", "MDX-UVR Vocal Model Voc_FT"]
Demucs = 'off'#@param ["off","demucs_extra"]

#@markdown ---
#@markdown ### Parameters
denoise = False #@param {type:"boolean"}
normalise = True #@param {type:"boolean"}
#getting values from model_data.json related to ONNX var (model folder name)
amplitude_compensation = model_data[ONNX]["compensate"]
dim_f = model_data[ONNX]["mdx_dim_f_set"]
dim_t = model_data[ONNX]["mdx_dim_t_set"]
n_fft = model_data[ONNX]["mdx_n_fft_scale_set"]

mixing_algorithm = 'max_mag' #@param ["default","min_mag","max_mag"]
chunks = 55 #@param {type:"slider", min:1, max:55, step:1}
shifts = 10 #@param {type:"slider", min:0, max:10, step:0.1}

##validate values
track = track if 'http' in track else tracks_path+track
normalise = '--normalise' if normalise else ''
denoise = '--denoise' if denoise else ''

if ONNX == 'off':
    pass
else:
    ONNX = 'onnx/'+ONNX
if Demucs == 'off':
    pass
else:
    Demucs = 'model/'+Demucs+'.th'
#@markdown ---
#@markdown ### Stems
bass = False #@param {type:"boolean"}
drums = False #@param {type:"boolean"}
others = False #@param {type:"boolean"}
vocals = True #@param {type:"boolean"}
#@markdown ---
#@markdown ### Invert stems to mixture
invert_bass = False #@param {type:"boolean"}
invert_drums = False #@param {type:"boolean"}
invert_others = False #@param {type:"boolean"}
invert_vocals = True #@param {type:"boolean"}
invert_stems = []
stems = []
if bass:
    stems.append('b')
if drums:
    stems.append('d')
if others:
    stems.append('o')
if vocals:
    stems.append('v')

invert_stems = []
if invert_bass:
    invert_stems.append('b')
if invert_drums:
    invert_stems.append('d')
if invert_others:
    invert_stems.append('o')
if invert_vocals:
    invert_stems.append('v')

margin = 44100

###
# incompatibilities
###

console(f"python main.py --n_fft {n_fft} --dim_f {dim_f} --dim_t {dim_t} --margin {margin} -i \"{track}\" --mixing {mixing_algorithm} --onnx \"{ONNX}\" --model {Demucs} --shifts {round(shifts)} --stems {''.join(stems)} --invert {''.join(invert_stems)} --chunks {chunks} --compensate {amplitude_compensation} {normalise} {denoise}")

<sup>Models provided are from [Kuielab](https://github.com/kuielab/mdx-net-submission/), [UVR](https://github.com/Anjok07/ultimatevocalremovergui/) and [Kim](https://github.com/KimberleyJensen/) <br> (you can support UVR [here](https://www.buymeacoffee.com/uvr5/vip-model-download-instructions) and [here](https://boosty.to/uvr)).</sup></br>
<sup>Original UVR notebook by [Audio Hacker](https://www.youtube.com/channel/UC0NiSV1jLMH-9E09wiDVFYw/), modified by Audio Separation community & then kalomaze (for RVC colab).</sup></br>
<sup>Big thanks to the [Audio Separation Discord](https://discord.gg/zeYU2Wzbgj) for helping me implement this in the colab.</sup></br>

##**UVR Colab Settings explanation**<br>

* **Mixing algorithm**: "max_mag" is for vocals and "min_mag" is for instrumentals. "Default" is a balance between the two and is required for Demucs enabled with vocal models.
* **Chunks**: Set it to 55 or 40 for less aggressive instrument disappearance. Set it to 1 for the best clarity. Use 10 for Demucs enabled or vocal models with tracks below 5:00 minutes. Higher numbers increase separation speed but may cause memory allocation errors.
* **Shifts**: Use a maximum of 10 for a slight increase in quality, but note that it significantly increases processing time. Shifts 5 give similar results.
* **Normalize**: Normalizes input and improves the separation process by reducing noise. Disable it if you have noisy hi-hats or large amplitude compensation.